<a id='top'></a>

# CSCI 3202, Spring 2020
# Assignment 3
# Due:  Wednesday 4 March 2020 by 11:59 PM

<br>

### Your name:

<br>




**Some things that might be useful**


In [1]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time
import heapq

<br>

---

## Problem 1 (35 points):  Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>

<a id='p2a'></a>

### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **2b** and **2d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)

        
class TicTacToe:
    
    def __init__(self, nrow=3, ncol=3, nwin=3, nexp=0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
#        moves = # insert your general list of nrow x ncol moves here
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        self.nexp = nexp

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''

        # your code goes here
        
        # Solution:
        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state

        
    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''        

        # your code goes here

        # Solution:
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0
        

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''

        # your code goes here
        
        # Solution:
        return state.utility!=0 or len(state.moves)==0    

    
    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''

        # your code goes here
        
        # Solution:
        return state.utility if player=='X' else -state.utility        
        
        
    def display(self):
        for row in range(1, self.nrow+1):
            for col in range(1, self.ncol+1):
                print(self.state.board.get((row, col), '.'), end=' ')
            print()
        
    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''

        # your code goes here

        # Solution:
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    #self.display()
                    return self.state.utility                



<br>


### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in Tic-Tac-Toe state argument'''

    
    # your code goes here...

    # Solution:
    possible_actions = game.state.moves
    random_move = possible_actions[np.random.randint(low=0, high=len(possible_actions))]
    
    return random_move

We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of Tic-Tac-Toe against one another, whoever goes first will win about 58% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.

**"Unit tests":** If you are wondering how close is close enough to 58%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 54-63%.

In [4]:
# Solution:
    
wins = 0
losses = 0
draws = 0
n_games = 1000
for _ in range(n_games):
    ttt = TicTacToe(3,3,3)
    outcome = ttt.play_game(random_player, random_player)
#    outcome = ttt.play_game(alphabeta_player, random_player)
#    outcome = ttt.play_game(alphabeta_player, alphabeta_player)
#    outcome = ttt.play_game(random_player, alphabeta_player)
#    print(outcome)
    if outcome==1:
        wins += 1
    elif outcome==0:
        draws += 1
    elif outcome==-1:
        losses += 1

print('Player 1: %wins={}, %draws={}, %losses={}'.format(100*wins/n_games, 100*draws/n_games, 100*losses/n_games))

Player 1: %wins=56.9, %draws=14.4, %losses=28.7


<br>


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **2b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [5]:
# Solution:
    
wins = 0
losses = 0
draws = 0
n_games = 1000
for _ in range(n_games):
    ttt = TicTacToe(4,4,4)
    outcome = ttt.play_game(random_player, random_player)
#    outcome = ttt.play_game(alphabeta_player, random_player)
#    outcome = ttt.play_game(alphabeta_player, alphabeta_player)
#    outcome = ttt.play_game(random_player, alphabeta_player)
#    print(outcome)
    if outcome==1:
        wins += 1
    elif outcome==0:
        draws += 1
    elif outcome==-1:
        losses += 1

print('Player 1: %wins={}, %draws={}, %losses={}'.format(100*wins/n_games, 100*draws/n_games, 100*losses/n_games))

Player 1: %wins=31.4, %draws=43.9, %losses=24.7


<br>


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [6]:
# Solution:
def random_player(game):
    '''A player that chooses a legal move at random.'''
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]


def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    '''search game approach to find best action, using alpha-beta pruning:
    alpha = best (highest) move found so far for Max
    beta  = best (lowest) move found so far for Min'''

    player = game.state.to_move

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, player)
        value = -float('inf')
        for a in state.moves:
            
            value = max(value, min_value(game.result(a, state), alpha, beta))
            if value >= beta:
                return value
            alpha = max(alpha, value)
        return value

    def min_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, player)
        value = float('inf')
        for a in state.moves:
            
            value = min(value, max_value(game.result(a, state), alpha, beta))
            if value <= alpha:
                return value
            beta = min(beta, value)
        return value

    # Body of alphabeta_cutoff_search:
    best_score = -float('inf')
    beta = float('inf')
    best_action = None
    for a in game.state.moves:
        
        value = min_value(game.result(a, game.state), best_score, beta)
        if value > best_score:
            best_score = value
            best_action = a
    return best_action

###################################
# def alphabeta_player(game):
#     return alphabeta_search(game)

# def alphabeta_search(game):
#     '''search game approach to find best action, using alpha-beta pruning:
#     alpha = best (highest) move found so far for Max
#     beta  = best (lowest) move found so far for Min'''

#     player = game.state.to_move

#     # Functions used by alphabeta
#     def max_value(state, alpha, beta):
#         if game.game_over(state):
#             return game.utility(state, player)
#         value = -float('inf')
#         for a in state.moves:
#             game.nexp += 1
#             value = max(value, min_value(game.result(a, state), alpha, beta))
#             if value >= beta:
#                 return value
#             alpha = max(alpha, value)
#         return value

#     def min_value(state, alpha, beta):
#         if game.game_over(state):
#             return game.utility(state, player)
#         value = float('inf')
#         for a in state.moves:
#             game.nexp += 1
#             value = min(value, max_value(game.result(a, state), alpha, beta))
#             if value <= alpha:
#                 return value
#             beta = min(beta, value)
#         return value

#     best_score = -float('inf')
#     beta = float('inf')
#     best_action = None
#     for a in game.state.moves:
#         game.nexp += 1
#         value = min_value(game.result(a, game.state), best_score, beta)
#         if value > best_score:
#             best_score = value
#             best_action = a
#     return best_action, game.nexp

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **only 10 games for each test**, and track the number of wins, draws and losses.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. 

In [7]:
# Solution:

tbeg = time()

wins = 0
losses = 0
draws = 0
n_games = 10
for _ in range(n_games):
    ttt = TicTacToe(3,3,3)
#    outcome = ttt.play_game(random_player, random_player)
#    outcome = ttt.play_game(alphabeta_player, random_player)
    outcome = ttt.play_game(alphabeta_player, alphabeta_player)

#    print(outcome)
    if outcome==1:
        wins += 1
    elif outcome==0:
        draws += 1
    elif outcome==-1:
        losses += 1

tend = time()

print('%wins={}, %draws={}, %losses={}'.format(100*wins/n_games, 100*draws/n_games, 100*losses/n_games))
print('Took {} seconds'.format(tend-tbeg))

%wins=0.0, %draws=100.0, %losses=0.0
Took 12.950598955154419 seconds


<br>


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, with and without pruning, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [8]:
# Solution:
#same code as above, but now including node expansion count

def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    '''search game approach to find best action, using alpha-beta pruning:
    alpha = best (highest) move found so far for Max
    beta  = best (lowest) move found so far for Min'''

    player = game.state.to_move

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, player)
        value = -float('inf')
        for a in state.moves:
            game.nexp += 1
            value = max(value, min_value(game.result(a, state), alpha, beta))
            if value >= beta:
                return value
            alpha = max(alpha, value)
        return value

    def min_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, player)
        value = float('inf')
        for a in state.moves:
            game.nexp += 1
            value = min(value, max_value(game.result(a, state), alpha, beta))
            if value <= alpha:
                return value
            beta = min(beta, value)
        return value

    best_score = -float('inf')
    beta = float('inf')
    best_action = None
    for a in game.state.moves:
        game.nexp += 1
        value = min_value(game.result(a, game.state), best_score, beta)
        if value > best_score:
            best_score = value
            best_action = a
    return best_action, game.nexp


def minimax_player(game):
    return minimax_search(game)

def minimax_search(game):
    '''same as alphabeta above, but no pruning'''

    player = game.state.to_move

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, player)
        value = -float('inf')
        for a in state.moves:
            game.nexp += 1
            value = max(value, min_value(game.result(a, state), alpha, beta))
##            if value >= beta:
##                return value
            alpha = max(alpha, value)
        return value

    def min_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, player)
        value = float('inf')
        for a in state.moves:
            game.nexp += 1
            value = min(value, max_value(game.result(a, state), alpha, beta))
##            if value <= alpha:
##                return value
            beta = min(beta, value)
        return value

    best_score = -float('inf')
    beta = float('inf')
    best_action = None
    for a in game.state.moves:
        game.nexp += 1
        value = min_value(game.result(a, game.state), best_score, beta)
        if value > best_score:
            best_score = value
            best_action = a
    return best_action, game.nexp

ttt = TicTacToe(3,3,3)
move, nexp_alphabeta = alphabeta_player(ttt)
print('Expanded by alpha-beta: {}'.format(nexp_alphabeta))
ttt = TicTacToe(3,3,3)
move, nexp_minimax = minimax_player(ttt)
print('Expanded by minimax: {}'.format(nexp_minimax))
print('Percent savings from pruning: {}'.format((nexp_minimax-nexp_alphabeta)/nexp_minimax))

Expanded by alpha-beta: 18296
Expanded by minimax: 549945
Percent savings from pruning: 0.9667312185764031


<br>

---

## Problem 2 (30 points):  Uncertainty

### (2a) 

Suppose you are deciding when to arrive at a party. There is some optimal time to arrive when the loss you feel, as measured by _awkwardness_, is minimized at 0. That is, at some particular time, it is not awkward at all to show up to the party. The awkwardness (loss) increases as you arrive too early or too late relative to this optimal time. What is a suitable loss function, $L(d, x)$, to model this situation? Include definitions for $d$ and $x$, consistent with the examples from this class. Use this loss function this weekend when you go out.

**Solution:** Many possible answers. Anything that had a minimum of 0 when $d-x$ and increases as $|d-x|$ is okay.

One possibility is quadratic loss: $ L(d, x) = (d-x)^2 $

Another is absolute value loss: $L(d, x) = |d-x| $

### (2b)

Suppose we have a situation where loss is given by the function $L(d, x) = 2(d-x)^2$. Set up, simplify, and evaluate integral(s) for the expected loss, $E_x[L(d, x)]$, where your prior beliefs regarding $x$ follow the distribution $f(x)$ given below. You may assume $f(x)=0$ for values of $x$ outside of the interval $[0, 3]$.

f(x) =  \begin{cases} 
      1/2 & 0 \leq x <1 \\
      3/8 & 1 \leq x <2 \\
      1/8 & 2 \leq x \leq 3 
   \end{cases}


**Solution:**

\begin{align*}
E_x[L(d,x)] &= \int_{-\infty}^{\infty} L(d,x)f(x) dx \\
            &= \int_0^1 2(d-x)^2 \cdot \frac{1}{2} dx + \int_1^2 2(d-x)^2 \cdot \frac{3}{8} dx + \int_2^3 2(d-x)^2 \cdot\frac{1}{8} dx \\
            &= \int_0^1 (d-x)^2 dx + \frac{3}{4} \int_1^2 (d-x)^2 dx + \frac{1}{4} \int_2^3 (d-x)^2 dx \\
            &= \left (d^2 x - dx^2 + \frac{1}{3} x^3 \right )|_0^1 + \frac{3}{4} \left (d^2 x - dx^2 + \frac{1}{3}x^3 \right )|_1^2 + \frac{1}{4} \left (d^2 x - dx^2+\frac{1}{3} x^3 \right )|_2^3 \\
            &= 2d^2 - \frac{18}{4}d + \frac{11}{3}
\end{align*}

### (2c) 

Suppose our expected loss is represented by the function $E_x[(L(d,x)]=(2-d)^2+2$, and our prior beliefs regarding $x$ are given by the distribution $f(x)$ from part b.

- Calculate Bayes' Decision, $d_{Bayes}$.

- Calculate the Expected Value of Including Uncertainty, EVIU. Suppose that if we ignore uncertainty, our best guess for what decision to make is the median of $x$ (under our prior $f(x)$).

**Solution:**

Bayes' Decision minimizes the expected loss, $d_{Bayes} = 2$.

$EVIU = E_x[L(d_{iu},x)]-E_x[L(d_{Bayes},x)]$ and $d_{iu}=median(x)=1$, (half the probability mass on either side.)

\begin{align*}
EVIU &= E_x[L(d_{iu},x)]-E_x[L(d_{Bayes},x)] \\
     &= [(2-1)^2+2]-[(2-2)^2+2] \\
     &= 1 \\
\end{align*}



<br>

## Problem 3 (35 points): Maximizing some objective function with a Genetic Algorithm

Suppose we are trying to figure out a cookie recipe, but can't quite remember how much of each ingredient we need.

So we want to maximize the following objective function corresponding to how close we are to this recipe:

* 3/4 cup granulated sugar (36 tsp)
* 3/4 cup packed brown sugar  (36 tsp)
* 1 cup butter (48 tsp)
* 1 teaspoon vanilla (1 tsp)
* 1 egg
* 2 1/4 cups flour (108 tsp)
* 1 teaspoon baking soda (1 tsp)
* 1/2 teaspoon salt (0.5 tsp)
* 1 package (12 ounces) chocolate chips (2 cups) (96 tsp)

In [9]:
target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]

An example starting state for a member of our population might look like: $state = [30, 30, 40, 1, 0, 100, 0.5, 0.5, 100]$

### (3a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [10]:
def recipe_success(state):
    target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]
    
    fitness = 0
    for i in range(len(target)):
        if target[i] == state[i]:
            fitness += 100
        else:
            fitness = fitness
    return fitness

In [11]:
# testing out the function
# which is written to maximize

state1 = [36,36,48,1,1,108,0.5,0.5,100]
recipe_success(state1)

700

### (3b) 

Using our in class notebook "CSCI3202_GeneticAlgorithm.ipynb" as your guide, write a genetic algorithm that starts with a population of 10 randomly generated "recipes/states/members" and uses the objective function you wrote in **(3a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from numbers between 0 and 108 (half step intervals might be helpful since the recipe requires 1/2 tsp salt)
- Allow for mutations in your population with an overall probability of mutation set to p = 0.1
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 200 iterations ("generations"). Do you hit your target?

In [12]:
# Solution

class problem:
    
    def __init__(self, initial_population, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)
        '''
        self.population = initial_population
        self.initial_population = initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.n_pop = len(initial_population)
        self.n_dna = len(initial_population[0])
        self.fitness_goal = fitness_goal

    def fitness(self):
        '''
        calculate each population member's probability of being selected for
        reproduction based on performance on objective function
        '''
        performance = []
        for k in range(self.n_pop):
            performance.append(self.objective_function(self.population[k]))
        
            
        total = sum(performance)
        p_reproduce = [perf/total for perf in performance]
        


        return p_reproduce
        
    def reproduce(self, parent1, parent2):
        # last DNA snippet from parent1:
        split = np.random.randint(low=1, high=self.n_dna)
        child = parent1[:split] + parent2[split:]
        return child

    def mutate(self, child):
        # which gene to mutate?
        gene = np.random.randint(low=0, high=self.n_dna)
        if child[gene]==0:
            child[gene]=1
        elif child[gene]==1:
            child[gene]=0
        return child
    
def genetic_algorithm(problem, n_iter):
    
    for t in range(n_iter):
        
        new_generation = []
        
        for k in range(problem.n_pop):
            
            # select for reproduction
            p_reproduce = problem.fitness()
            #ind_parents = np.random.choice(range(0,problem.n_pop), size=2, replace=False)
            ind_parents = np.random.choice(range(0,problem.n_pop), size=2, p=p_reproduce, replace=False)
            parent1, parent2 = problem.population[ind_parents[0]], problem.population[ind_parents[1]]
            
            
            # reproduce
            child = problem.reproduce(parent1, parent2)
            
            # mutate
            l_mutate = np.random.choice([True, False], p=[problem.p_mutate, 1-problem.p_mutate])
            if l_mutate:
                child = problem.mutate(child)
            
            # add to new generation
            new_generation.append(child)
        
        # set problem.population = new generation
        problem.population = new_generation
        
        # exit criterion check
        performance = [problem.objective_function(member) for member in problem.population]
        
        best_member = max(zip(performance, problem.population))
        
        if best_member[0] >= problem.fitness_goal:
            return best_member

    print('reached n_iter', best_member)

    return False 

In [13]:
# We need to create an initial population!
initial_pop_number = 100

gene_pool = np.arange(0,108,0.5)
num_ingredients = 9

initial_population = []

for ii in range(initial_pop_number):
    
    new_member = []
    for i in range(num_ingredients):
        gene = np.random.choice(gene_pool, replace = True)
        new_member.append(gene)
    initial_population.append(new_member)
    
# print(initial_population)

In [14]:
genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=900.0, 
                          mutation_probability=0.1, 
                          objective_function=recipe_success)
out = genetic_algorithm(genetic_problem, 600)
print(out)

reached n_iter (500, [90.5, 36.0, 48.0, 1.0, 1.0, 45.5, 62.0, 46.5, 96.0])
False


### (3c)

Report the following:
- How many generations did it take to hit the goal?
- If you change the initial population size to 100, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation to 0.2, does that affect the number of generations it takes to achieve the goal recipe?

Alternate questions to answer if Target not hit:
- Report whether you minimized of maximized the objective function and whether that led to any major changes in how you designed the probability of reproduction. A couple sentences here is fine.
 
- Report how many ingredients you ended up matching. e.g. target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96], perhaps your algorithm achieved [36, 42, 8, 1, 1, 100, 56, 0.5, 0], then you would have matched 4 of the ingredient values.
 
- Report how many iterations you tried in order to get this answer. (Don't burn up your machine in the process)

In [15]:
# There are many possible solutions for this part.

I went up to 600 generations and never hit the target exactly. Above you can see that I matched 5 ingredients.I set up the objective function to be maximized.

Changing the initial population size did not seem to help much. 

Changing the mutation probability also did not help a significant amount.